In [1]:
# NEED TO ADD MAX TEAM CAPACITY CONSTRAINT

import pandas as pd
import numpy as np

# import the required classes and definitions from Python-MIP
from mip import Model, xsum, maximize, BINARY, INTEGER

Using Python-MIP package version 1.5.1


In [2]:
#sort the top500 by element_type so model can fill correct number of each type

top500 = pd.read_csv('top500.csv')
top500sorted = top500.sort_values('element_type')


#lists for the dream team model, sorted by position
eltype = list(top500sorted['element_type'])
form = list(top500sorted['form'])
score = list(top500sorted['total_points'])
price = list(top500sorted['now_cost'])
form_score = list(pd.Series(form) * pd.Series(score))


In [3]:
top500sorted.shape

(500, 53)

In [4]:
#make a list of our main columns of interest
simple_columns = ['web_name', 'element_type', 'total_points','now_cost', 'form',
                 'goals_scored', 'assists', 'clean_sheets', 'team', 'selected_by_percent']

#make a smaller df for easy viewing of dream_team
top500simple = top500sorted[simple_columns]
top500simple.shape

(500, 10)

In [5]:
#find cutoffs for the element_types

eltype_counts = top500sorted['element_type'].value_counts().sort_index()

cutoff1 = eltype_counts[1]
cutoff2 = cutoff1 + eltype_counts[2]
cutoff3 = cutoff2 + eltype_counts[3]
cutoff4 = cutoff3 + eltype_counts[4]

print(cutoff1, cutoff2, cutoff3, cutoff4)
eltype_counts

50 222 430 500


1     50
2    172
3    208
4     70
Name: element_type, dtype: int64

In [13]:
# The model

# define the problem data, change the profit list here
profit = score
weight = price
c = 830
n = len(weight)

#create an empty maximization
m = Model('team_select')

#add the binary decision variables to model m and store their references in a list x
x = [m.add_var(var_type=BINARY) for i in range(n)]

#define the objective function of this model 
m.objective = maximize(xsum(profit[i] * x[i] for i in range(n)))

#add the capacity constraints
m += xsum(weight[i] * x[i] for i in range(n)) <= c

m += xsum(x) <= 11

m += xsum(eltype[i] * x[i] for i in range(cutoff1)) >= 1
m += xsum(eltype[i] * x[i] for i in range(cutoff1)) <= 1

m += xsum(eltype[i] * x[i] for i in range(cutoff1,cutoff2)) >= 6
m += xsum(eltype[i] * x[i] for i in range(cutoff1,cutoff2)) <= 10

m += xsum(eltype[i] * x[i] for i in range(cutoff2,cutoff3)) >= 6
m += xsum(eltype[i] * x[i] for i in range(cutoff2,cutoff3)) <= 15

m += xsum(eltype[i] * x[i] for i in range(cutoff3,cutoff4)) >= 4
m += xsum(eltype[i] * x[i] for i in range(cutoff3,cutoff4)) <= 12

#Optimize the model
m.optimize()

# Compute the solution, a list of the selected items
selected = [i for i in range(n) if x[i].x >= 0.99]
print('selected items: {}'.format(selected))

selected items: [36, 174, 177, 183, 374, 376, 390, 393, 430, 448, 499]


In [14]:
dream_team = top500simple.iloc[selected,:]
dream_team

,web_name,element_type,total_points,now_cost,form,goals_scored,assists,clean_sheets,team,selected_by_percent
22,Ryan,1,55,47,5.0,0,0,4,4,15.7
6,Lundstram,2,70,48,7.8,3,1,5,15,41.1
14,Pereira,2,61,63,5.2,2,0,5,9,14.6
23,Söyüncü,2,55,48,7.5,1,1,5,9,11.7
11,David Silva,3,61,76,5.2,3,5,6,11,8.6
10,Maddison,3,62,74,7.0,4,3,4,9,19.9
1,Mané,3,83,120,6.5,7,4,3,10,34.5
3,De Bruyne,3,76,102,3.2,2,10,5,11,40.2
2,Abraham,4,81,79,6.8,10,2,4,6,50.0
12,Pukki,4,61,67,1.8,6,2,1,14,22.0


In [15]:
sum(dream_team['element_type'])

31

In [16]:
sum(dream_team['now_cost'])

819

In [17]:
sum(dream_team['total_points'])

757